In [22]:

import pandas as pd
import pickle
timestamp = "run_2020_04_20_19_20_19_ALL_r2n07"
epoch = "249"
filename = f"/Users/elliotschumacher/Dropbox/git/clel/results/{timestamp}/eval_{epoch}.xlsx"
data_file = "/Users/elliotschumacher/Dropbox/git/clel/datasets/LDC2019T02/ENG/tac_kbp_2015_tedl_training_gold_standard_entity_mentions.tab"

df = pd.read_excel(filename)
header_list = ["system_id", "query_id", "mention_string", "doc_id_offsets", "link_id", "entity_type", "mention_type",
               "confidence", "web_search", "wiki_text", "unknown"]
data = pd.read_csv(data_file, sep="\t", names=header_list, header=None,)



data = data[~data['link_id'].str.startswith('NIL')]
data['lang'] =  data.apply (lambda row: row['doc_id_offsets'][:3], axis=1)

popularity = pd.pivot_table(data, index=["link_id", "lang"],
                            values="query_id",
                             aggfunc="count").reset_index().sort_values(by="query_id", ascending=False)
popularity

,link_id,lang,query_id
1885,m.09c7w0,CMN,1515
2099,m.0d05w3,CMN,824
963,m.03h64,CMN,650
1886,m.09c7w0,ENG,628
2093,m.0d04z6,CMN,556
...,...,...,...
1115,m.047drb0,CMN,1
1113,m.047c82x,ENG,1
1112,m.0478__m,ENG,1
1108,m.046tsq,CMN,1


In [37]:
training_total = 0
eval_total = 0
training_entities = set()
eval_entites = {}

eval_query_ids = set()
training_query_ids = set()

for link_id in data['link_id'].unique():
    link_id_counts = popularity[popularity['link_id'] == link_id]
    if len(link_id_counts) == 3:
        min_count = link_id_counts['query_id'].min()
    else:
        min_count = 0
    these_queries = data[data["link_id"] == link_id]
    if min_count > 0:
        eval_entites[link_id] = min_count
        eval_total += min_count
        for lang in ['ENG', 'SPA', 'CMN']:
            these_queries_lang = these_queries[these_queries['lang'] == lang].sample(min_count)
            eval_query_ids.update(these_queries_lang['query_id'])
    else:
        training_total += link_id_counts['query_id'].sum()
        training_entities.add(link_id)
        training_query_ids.update(these_queries['query_id'])
print(eval_entites)
print(eval_total)
print(training_total)
print(len(training_query_ids))
print(len(eval_query_ids))
with open(f"/Users/elliotschumacher/Dropbox/git/clel/datasets/LDC2019T02/query_splits/eval_query_ids.pkl", 'wb') as f:
    pickle.dump(eval_query_ids, f)
with open(f"/Users/elliotschumacher/Dropbox/git/clel/datasets/LDC2019T02/query_splits/training_query_ids.pkl", 'wb') as f:
    pickle.dump(training_query_ids, f)    

{'m.0f1lfn': 84, 'm.0qtngg8': 19, 'm.0hzlz': 29, 'm.02mjmr': 98, 'm.01gcjq': 12, 'm.0c0cs': 5, 'm.0cv_2': 2, 'm.0c0bf': 2, 'm.07ylj': 8, 'm.07t65': 15, 'm.0d3qd0': 6, 'm.09c7w0': 263, 'm.0d04z6': 70, 'm.07c5l': 1, 'm.05qx1': 1, 'm.01cx_': 34, 'm.05k7sb': 7, 'm.0t4r0zs': 59, 'm.04rwx': 2, 'm.0d05w3': 9, 'm.0d6hn': 3, 'm.09k0f': 5, 'm.0d6qjf': 2, 'm.04pnx': 2, 'm.0261m': 3, 'm.0n9sd9l': 2, 'm.09j2x': 4, 'm.0hzrl7z': 41, 'm.07b_l': 11, 'm.0j0k': 1, 'm.0d05q4': 58, 'm.05gnf': 1, 'm.02czjf': 1, 'm.01kk8b': 3, 'm.01skqzw': 3, 'm.01crd5': 1, 'm.02_286': 6, 'm.0bwh6': 3, 'm.07wh1': 1, 'm.07vsl': 10, 'm.03_r3': 1, 'm.05fz6q': 37, 'm.06bnz': 78, 'm.0cv92n': 1, 'm.04swd': 20, 'm.0fm2h': 77, 'm.03shp': 57, 'm.027x630': 106, 'm.03spz': 102, 'm.04zxt7x': 1, 'm.0g9z94d': 1, 'm.01dcs1': 8, 'm.0cf7z6': 3, 'm.07wbk': 14, 'm.04wsz': 6, 'm.0d075m': 10, 'm.039rwf': 2, 'm.0b3wk': 3, 'm.081sq': 9, 'm.02j9z': 6, 'm.07t21': 13, 'm.07t58': 2, 'm.06mzp': 5, 'm.0c1pjf': 3, 'm.07ssc': 25, 'm.0f8l9c': 8, 'm.0345h':